<a href="https://www.kaggle.com/nadernarcisse/covid-19-prediction-days-10-20-30-40-50?scriptVersionId=82753366" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/07-18-2020.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/07-18-2020.csv')

In [3]:
cols = confirmed_df.keys()

In [4]:
confirmed = confirmed_df.loc[:, cols[4]:cols[-1]]
deaths = deaths_df.loc[:, cols[4]:cols[-1]]
recoveries = recoveries_df.loc[:, cols[4]:cols[-1]]

In [5]:
dates = confirmed.keys()
world_cases = []
total_deaths = [] 
mortality_rate = []
recovery_rate = [] 
total_recovered = [] 
total_active = [] 

china_cases = [] 
italy_cases = []
us_cases = [] 
spain_cases = [] 
france_cases = [] 
germany_cases = [] 
uk_cases = [] 
russia_cases = [] 
brazil_cases = []
india_cases = []
peru_cases = [] 

china_deaths = [] 
italy_deaths = []
us_deaths = [] 
spain_deaths = [] 
france_deaths = [] 
germany_deaths = [] 
uk_deaths = [] 
russia_deaths = []
brazil_deaths = [] 
india_deaths = []
peru_deaths = []

china_recoveries = [] 
italy_recoveries = []
us_recoveries = [] 
spain_recoveries = [] 
france_recoveries = [] 
germany_recoveries = [] 
uk_recoveries = [] 
russia_recoveries = [] 
brazil_recoveries = [] 
india_recoveries = [] 
peru_recoveries = [] 

for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # confirmed, deaths, recovered, and active
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)

    # case studies 
    china_cases.append(confirmed_df[confirmed_df['Country/Region']=='China'][i].sum())
    italy_cases.append(confirmed_df[confirmed_df['Country/Region']=='Italy'][i].sum())
    us_cases.append(confirmed_df[confirmed_df['Country/Region']=='US'][i].sum())
    spain_cases.append(confirmed_df[confirmed_df['Country/Region']=='Spain'][i].sum())
    france_cases.append(confirmed_df[confirmed_df['Country/Region']=='France'][i].sum())
    germany_cases.append(confirmed_df[confirmed_df['Country/Region']=='Germany'][i].sum())
    uk_cases.append(confirmed_df[confirmed_df['Country/Region']=='United Kingdom'][i].sum())
    russia_cases.append(confirmed_df[confirmed_df['Country/Region']=='Russia'][i].sum())
    brazil_cases.append(confirmed_df[confirmed_df['Country/Region']=='Brazil'][i].sum())
    india_cases.append(confirmed_df[confirmed_df['Country/Region']=='India'][i].sum())
    peru_cases.append(confirmed_df[confirmed_df['Country/Region']=='Peru'][i].sum())
    
    
    china_deaths.append(deaths_df[deaths_df['Country/Region']=='China'][i].sum())
    italy_deaths.append(deaths_df[deaths_df['Country/Region']=='Italy'][i].sum())
    us_deaths.append(deaths_df[deaths_df['Country/Region']=='US'][i].sum())
    spain_deaths.append(deaths_df[deaths_df['Country/Region']=='Spain'][i].sum())
    france_deaths.append(deaths_df[deaths_df['Country/Region']=='France'][i].sum())
    germany_deaths.append(deaths_df[deaths_df['Country/Region']=='Germany'][i].sum())
    uk_deaths.append(deaths_df[deaths_df['Country/Region']=='United Kingdom'][i].sum())
    russia_deaths.append(deaths_df[deaths_df['Country/Region']=='Russia'][i].sum())
    brazil_deaths.append(deaths_df[deaths_df['Country/Region']=='Brazil'][i].sum())
    india_deaths.append(deaths_df[deaths_df['Country/Region']=='India'][i].sum())
    peru_deaths.append(deaths_df[deaths_df['Country/Region']=='Peru'][i].sum())
    
    china_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='China'][i].sum())
    italy_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Italy'][i].sum())
    us_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='US'][i].sum())
    spain_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Spain'][i].sum())
    france_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='France'][i].sum())
    germany_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Germany'][i].sum())
    uk_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='United Kingdom'][i].sum())
    russia_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Russia'][i].sum())
    brazil_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Brazil'][i].sum())
    india_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='India'][i].sum())
    peru_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Peru'][i].sum())

In [6]:
dates = confirmed.keys()
world_cases = []
total_deaths = [] 
mortality_rate = []
recovery_rate = [] 
total_recovered = [] 
total_active = [] 

china_cases = [] 
italy_cases = []
us_cases = [] 
spain_cases = [] 
france_cases = [] 
germany_cases = [] 
uk_cases = [] 
russia_cases = [] 
brazil_cases = []
india_cases = []
peru_cases = [] 

china_deaths = [] 
italy_deaths = []
us_deaths = [] 
spain_deaths = [] 
france_deaths = [] 
germany_deaths = [] 
uk_deaths = [] 
russia_deaths = []
brazil_deaths = [] 
india_deaths = []
peru_deaths = []

china_recoveries = [] 
italy_recoveries = []
us_recoveries = [] 
spain_recoveries = [] 
france_recoveries = [] 
germany_recoveries = [] 
uk_recoveries = [] 
russia_recoveries = [] 
brazil_recoveries = [] 
india_recoveries = [] 
peru_recoveries = [] 

for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # confirmed, deaths, recovered, and active
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)

    # case studies 
    china_cases.append(confirmed_df[confirmed_df['Country/Region']=='China'][i].sum())
    italy_cases.append(confirmed_df[confirmed_df['Country/Region']=='Italy'][i].sum())
    us_cases.append(confirmed_df[confirmed_df['Country/Region']=='US'][i].sum())
    spain_cases.append(confirmed_df[confirmed_df['Country/Region']=='Spain'][i].sum())
    france_cases.append(confirmed_df[confirmed_df['Country/Region']=='France'][i].sum())
    germany_cases.append(confirmed_df[confirmed_df['Country/Region']=='Germany'][i].sum())
    uk_cases.append(confirmed_df[confirmed_df['Country/Region']=='United Kingdom'][i].sum())
    russia_cases.append(confirmed_df[confirmed_df['Country/Region']=='Russia'][i].sum())
    brazil_cases.append(confirmed_df[confirmed_df['Country/Region']=='Brazil'][i].sum())
    india_cases.append(confirmed_df[confirmed_df['Country/Region']=='India'][i].sum())
    peru_cases.append(confirmed_df[confirmed_df['Country/Region']=='Peru'][i].sum())
    
    
    china_deaths.append(deaths_df[deaths_df['Country/Region']=='China'][i].sum())
    italy_deaths.append(deaths_df[deaths_df['Country/Region']=='Italy'][i].sum())
    us_deaths.append(deaths_df[deaths_df['Country/Region']=='US'][i].sum())
    spain_deaths.append(deaths_df[deaths_df['Country/Region']=='Spain'][i].sum())
    france_deaths.append(deaths_df[deaths_df['Country/Region']=='France'][i].sum())
    germany_deaths.append(deaths_df[deaths_df['Country/Region']=='Germany'][i].sum())
    uk_deaths.append(deaths_df[deaths_df['Country/Region']=='United Kingdom'][i].sum())
    russia_deaths.append(deaths_df[deaths_df['Country/Region']=='Russia'][i].sum())
    brazil_deaths.append(deaths_df[deaths_df['Country/Region']=='Brazil'][i].sum())
    india_deaths.append(deaths_df[deaths_df['Country/Region']=='India'][i].sum())
    peru_deaths.append(deaths_df[deaths_df['Country/Region']=='Peru'][i].sum())
    
    china_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='China'][i].sum())
    italy_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Italy'][i].sum())
    us_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='US'][i].sum())
    spain_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Spain'][i].sum())
    france_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='France'][i].sum())
    germany_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Germany'][i].sum())
    uk_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='United Kingdom'][i].sum())
    russia_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Russia'][i].sum())
    brazil_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Brazil'][i].sum())
    india_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='India'][i].sum())
    peru_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Peru'][i].sum())

In [7]:
days_since_1_22 = np.array([i for i in range(len(dates))]).reshape(-1, 1)
world_cases = np.array(world_cases).reshape(-1, 1)
total_deaths = np.array(total_deaths).reshape(-1, 1)
total_recovered = np.array(total_recovered).reshape(-1, 1)

In [8]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]

In [9]:
start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

# Confirmed Cases

In [10]:
X_train_confirmed00X, X_test_confirmed00X, y_train_confirmed00X, y_test_confirmed00X = train_test_split(days_since_1_22[10:], world_cases[10:], test_size=0.12, shuffle=False)
# svm_confirmed = svm_search.best_estimator_
svm_confirmed00X = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed00X.fit(X_train_confirmed00X, y_train_confirmed00X)
svm_pred00X = svm_confirmed00X.predict(future_forcast)

# check against testing data
svm_test_pred00X = svm_confirmed00X.predict(X_test_confirmed00X)
print('MAE:', mean_absolute_error(svm_test_pred00X, y_test_confirmed00X))
print('MSE:',mean_squared_error(svm_test_pred00X, y_test_confirmed00X))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_confirmed00X = poly.fit_transform(X_train_confirmed00X)
poly_X_test_confirmed00X = poly.fit_transform(X_test_confirmed00X)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00X = PolynomialFeatures(degree=4)
bayesian_poly_X_train_confirmed00X = bayesian_poly00X.fit_transform(X_train_confirmed00X)
bayesian_poly_X_test_confirmed00X = bayesian_poly00X.fit_transform(X_test_confirmed00X)
bayesian_poly_future_forcast00X = bayesian_poly00X.fit_transform(future_forcast)

# polynomial regression
linear_model00X = LinearRegression(normalize=True, fit_intercept=False)
linear_model00X.fit(poly_X_train_confirmed00X, y_train_confirmed00X)
test_linear_pred00X = linear_model00X.predict(poly_X_test_confirmed00X)
linear_pred00X = linear_model00X.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00X, y_test_confirmed00X))
print('MSE:',mean_squared_error(test_linear_pred00X, y_test_confirmed00X))

print(linear_model00X.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00X = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00X = BayesianRidge(fit_intercept=False)
bayesian_search00X = RandomizedSearchCV(bayesian00X, bayesian_grid00X, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00X.fit(bayesian_poly_X_train_confirmed00X, y_train_confirmed00X)

bayesian_search00X.best_params_

bayesian_confirmed = bayesian_search00X.best_estimator_
test_bayesian_pred00X = bayesian_confirmed.predict(bayesian_poly_X_test_confirmed00X)
bayesian_pred00X = bayesian_confirmed.predict(bayesian_poly_future_forcast00X)
print('MAE:', mean_absolute_error(test_bayesian_pred00X, y_test_confirmed00X))
print('MSE:',mean_squared_error(test_bayesian_pred00X, y_test_confirmed00X))

MAE: 99202302.24803534
MSE: 1.0475884082174624e+16
MAE: 13223509.393490056
MSE: 243349566896717.25
[[-4.56043457e+06  2.17993297e+05 -2.68354036e+03  1.57647365e+01
  -2.81402807e-02  1.68020197e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 21748762.528555054
MSE: 597671707020268.9


[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    2.3s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
#10 Confirmed
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, world_cases, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00X, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00X, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00X, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Confirmed Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [12]:
X_train_confirmed00XX, X_test_confirmed00XX, y_train_confirmed00XX, y_test_confirmed00XX = train_test_split(days_since_1_22[20:], world_cases[20:], test_size=0.12, shuffle=False)
# svm_confirmed = svm_search.best_estimator_
svm_confirmed00XX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed00XX.fit(X_train_confirmed00XX, y_train_confirmed00XX)
svm_pred00XX = svm_confirmed00XX.predict(future_forcast)

# check against testing data
svm_test_pred00XX = svm_confirmed00XX.predict(X_test_confirmed00XX)
print('MAE:', mean_absolute_error(svm_test_pred00XX, y_test_confirmed00XX))
print('MSE:',mean_squared_error(svm_test_pred00XX, y_test_confirmed00XX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_confirmed00XX = poly.fit_transform(X_train_confirmed00XX)
poly_X_test_confirmed00XX = poly.fit_transform(X_test_confirmed00XX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00XX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_confirmed00XX = bayesian_poly00XX.fit_transform(X_train_confirmed00XX)
bayesian_poly_X_test_confirmed00XX = bayesian_poly00XX.fit_transform(X_test_confirmed00XX)
bayesian_poly_future_forcast00XX = bayesian_poly00XX.fit_transform(future_forcast)

# polynomial regression
linear_model00XX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00XX.fit(poly_X_train_confirmed00XX, y_train_confirmed00XX)
test_linear_pred00XX = linear_model00X.predict(poly_X_test_confirmed00XX)
linear_pred00XX = linear_model00X.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00XX, y_test_confirmed00XX))
print('MSE:',mean_squared_error(test_linear_pred00XX, y_test_confirmed00XX))

print(linear_model00XX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00XX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00XX = BayesianRidge(fit_intercept=False)
bayesian_search00XX = RandomizedSearchCV(bayesian00XX, bayesian_grid00XX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00XX.fit(bayesian_poly_X_train_confirmed00XX, y_train_confirmed00XX)

bayesian_search00XX.best_params_

bayesian_confirmed = bayesian_search00XX.best_estimator_
test_bayesian_pred00XX = bayesian_confirmed.predict(bayesian_poly_X_test_confirmed00XX)
bayesian_pred00XX = bayesian_confirmed.predict(bayesian_poly_future_forcast00XX)
print('MAE:', mean_absolute_error(test_bayesian_pred00XX, y_test_confirmed00XX))
print('MSE:',mean_squared_error(test_bayesian_pred00XX, y_test_confirmed00XX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 97872562.39819096
MSE: 1.0186637598723032e+16
MAE: 13375612.07126649
MSE: 246310364835562.16
[[-6.94505901e+06  2.77813283e+05 -3.16794130e+03  1.74612805e+01
  -3.08154442e-02  1.83603843e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 21591087.719026465
MSE: 588209066363368.4


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
#20 confirmed
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, world_cases, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00XX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00XX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00XX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Confirmed Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [14]:
X_train_confirmed00XXX, X_test_confirmed00XXX, y_train_confirmed00XXX, y_test_confirmed00XXX = train_test_split(days_since_1_22[30:], world_cases[30:], test_size=0.12, shuffle=False)
# svm_confirmed = svm_search.best_estimator_
svm_confirmed00XXX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed00XXX.fit(X_train_confirmed00XXX, y_train_confirmed00XXX)
svm_pred00XXX = svm_confirmed00XXX.predict(future_forcast)

# check against testing data
svm_test_pred00XXX = svm_confirmed00XXX.predict(X_test_confirmed00XXX)
print('MAE:', mean_absolute_error(svm_test_pred00XXX, y_test_confirmed00XXX))
print('MSE:',mean_squared_error(svm_test_pred00XXX, y_test_confirmed00XXX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_confirmed00XXX = poly.fit_transform(X_train_confirmed00XXX)
poly_X_test_confirmed00XXX = poly.fit_transform(X_test_confirmed00XXX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00XXX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_confirmed00XXX = bayesian_poly00XXX.fit_transform(X_train_confirmed00XXX)
bayesian_poly_X_test_confirmed00XXX = bayesian_poly00XXX.fit_transform(X_test_confirmed00XXX)
bayesian_poly_future_forcast00XXX = bayesian_poly00XXX.fit_transform(future_forcast)

# polynomial regression
linear_model00XXX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00XXX.fit(poly_X_train_confirmed00XXX, y_train_confirmed00XXX)
test_linear_pred00XXX = linear_model00X.predict(poly_X_test_confirmed00XXX)
linear_pred00XXX = linear_model00XXX.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00XXX, y_test_confirmed00XXX))
print('MSE:',mean_squared_error(test_linear_pred00XXX, y_test_confirmed00XXX))

print(linear_model00XXX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00XXX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00XXX = BayesianRidge(fit_intercept=False)
bayesian_search00XXX = RandomizedSearchCV(bayesian00XXX, bayesian_grid00XXX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00XXX.fit(bayesian_poly_X_train_confirmed00XXX, y_train_confirmed00XXX)

bayesian_search00XXX.best_params_

bayesian_confirmed = bayesian_search00XXX.best_estimator_
test_bayesian_pred00XXX = bayesian_confirmed.predict(bayesian_poly_X_test_confirmed00XXX)
bayesian_pred00XXX = bayesian_confirmed.predict(bayesian_poly_future_forcast00XXX)
print('MAE:', mean_absolute_error(test_bayesian_pred00XXX, y_test_confirmed00XXX))
print('MSE:',mean_squared_error(test_bayesian_pred00XXX, y_test_confirmed00XXX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 95186974.0266051
MSE: 9634897984486534.0
MAE: 13529132.74936805
MSE: 249340314746818.88
[[-9.91601743e+06  3.47096364e+05 -3.70503372e+03  1.92870525e+01
  -3.36312856e-02  1.99722708e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 21436785.466262218
MSE: 578997714355326.2


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
#30 confirmed
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, world_cases, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00XXX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00XXX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00XXX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Confirmed Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [16]:
X_train_confirmed00LX, X_test_confirmed00LX, y_train_confirmed00LX, y_test_confirmed00LX = train_test_split(days_since_1_22[40:], world_cases[40:], test_size=0.12, shuffle=False)
# svm_confirmed = svm_search.best_estimator_
svm_confirmed00LX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed00LX.fit(X_train_confirmed00LX, y_train_confirmed00LX)
svm_pred00LX = svm_confirmed00LX.predict(future_forcast)

# check against testing data
svm_test_pred00LX = svm_confirmed00LX.predict(X_test_confirmed00LX)
print('MAE:', mean_absolute_error(svm_test_pred00LX, y_test_confirmed00LX))
print('MSE:',mean_squared_error(svm_test_pred00LX, y_test_confirmed00LX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_confirmed00LX = poly.fit_transform(X_train_confirmed00LX)
poly_X_test_confirmed00LX = poly.fit_transform(X_test_confirmed00LX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00LX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_confirmed00LX = bayesian_poly00LX.fit_transform(X_train_confirmed00LX)
bayesian_poly_X_test_confirmed00LX = bayesian_poly00LX.fit_transform(X_test_confirmed00LX)
bayesian_poly_future_forcast00LX = bayesian_poly00LX.fit_transform(future_forcast)

# polynomial regression
linear_model00LX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00LX.fit(poly_X_train_confirmed00LX, y_train_confirmed00LX)
test_linear_pred00LX = linear_model00LX.predict(poly_X_test_confirmed00LX)
linear_pred00LX = linear_model00LX.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00LX, y_test_confirmed00LX))
print('MSE:',mean_squared_error(test_linear_pred00LX, y_test_confirmed00LX))

print(linear_model00LX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00LX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00LX = BayesianRidge(fit_intercept=False)
bayesian_search00LX = RandomizedSearchCV(bayesian00LX, bayesian_grid00LX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00LX.fit(bayesian_poly_X_train_confirmed00LX, y_train_confirmed00LX)

bayesian_search00LX.best_params_

bayesian_confirmed = bayesian_search00LX.best_estimator_
test_bayesian_pred00LX = bayesian_confirmed.predict(bayesian_poly_X_test_confirmed00LX)
bayesian_pred00LX = bayesian_confirmed.predict(bayesian_poly_future_forcast00LX)
print('MAE:', mean_absolute_error(test_bayesian_pred00LX, y_test_confirmed00LX))
print('MSE:',mean_squared_error(test_bayesian_pred00LX, y_test_confirmed00LX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 93202754.92552763
MSE: 9216852228986556.0


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 17460143.543123186
MSE: 411682827961226.9
[[-1.32998181e+07  4.20275306e+05 -4.24303963e+03  2.10419268e+01
  -3.62447243e-02  2.14221654e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 21123422.280620344
MSE: 560642331359772.94


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
#40 confirmed
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, world_cases, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00LX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00LX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00LX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Confirmed Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [18]:
X_train_confirmed00L, X_test_confirmed00L, y_train_confirmed00L, y_test_confirmed00L = train_test_split(days_since_1_22[50:], world_cases[50:], test_size=0.12, shuffle=False)
# svm_confirmed = svm_search.best_estimator_
svm_confirmed00L = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed00L.fit(X_train_confirmed00L, y_train_confirmed00L)
svm_pred00L = svm_confirmed00L.predict(future_forcast)

# check against testing data
svm_test_pred00L = svm_confirmed00L.predict(X_test_confirmed00L)
print('MAE:', mean_absolute_error(svm_test_pred00L, y_test_confirmed00L))
print('MSE:',mean_squared_error(svm_test_pred00L, y_test_confirmed00L))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_confirmed00L = poly.fit_transform(X_train_confirmed00L)
poly_X_test_confirmed00L = poly.fit_transform(X_test_confirmed00L)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00L = PolynomialFeatures(degree=4)
bayesian_poly_X_train_confirmed00L = bayesian_poly00L.fit_transform(X_train_confirmed00L)
bayesian_poly_X_test_confirmed00L = bayesian_poly00L.fit_transform(X_test_confirmed00L)
bayesian_poly_future_forcast00L = bayesian_poly00L.fit_transform(future_forcast)

# polynomial regression
linear_model00L = LinearRegression(normalize=True, fit_intercept=False)
linear_model00L.fit(poly_X_train_confirmed00L, y_train_confirmed00L)
test_linear_pred00L = linear_model00L.predict(poly_X_test_confirmed00L)
linear_pred00L = linear_model00L.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00L, y_test_confirmed00L))
print('MSE:',mean_squared_error(test_linear_pred00L, y_test_confirmed00L))

print(linear_model00L.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00L = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00L = BayesianRidge(fit_intercept=False)
bayesian_search00L = RandomizedSearchCV(bayesian00L, bayesian_grid00L, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00L.fit(bayesian_poly_X_train_confirmed00L, y_train_confirmed00L)

bayesian_search00L.best_params_

bayesian_confirmed = bayesian_search00L.best_estimator_
test_bayesian_pred00L = bayesian_confirmed.predict(bayesian_poly_X_test_confirmed00L)
bayesian_pred00L = bayesian_confirmed.predict(bayesian_poly_future_forcast00L)
print('MAE:', mean_absolute_error(test_bayesian_pred00L, y_test_confirmed00L))
print('MSE:',mean_squared_error(test_bayesian_pred00L, y_test_confirmed00L))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 92005757.66692895
MSE: 8971296310124766.0
MAE: 18504283.79873988
MSE: 459558361442330.6
[[-1.69140363e+07  4.94779568e+05 -4.77502927e+03  2.27458001e+01
  -3.87533587e-02  2.28044228e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 20964688.91552766
MSE: 551542460840585.44


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
#50 confirmed
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, world_cases, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00L, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00L, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00L, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Confirmed Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [20]:
# Future predictions using SVM 
svm_pred00X_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'SVM Predicted # of Confirmed Cases Worldwide (Input 10)': np.round(svm_pred00X[:-1])})
svm_pred00X_Confirmed_sub.to_csv('svm_pred00X_Confirmed_sub.csv', index = False)
svm_pred00X_Confirmed_sub

,Date,SVM Predicted # of Confirmed Cases Worldwide (Input 10)
0,01/22/2020,12036222.0
1,01/23/2020,12036223.0
2,01/24/2020,12036231.0
3,01/25/2020,12036254.0
4,01/26/2020,12036298.0
...,...,...
701,12/23/2021,425698424.0
702,12/24/2021,427471260.0
703,12/25/2021,429249154.0
704,12/26/2021,431032113.0


In [21]:
linear_pred00X = linear_pred00X.reshape(1,-1)[0]
linear_pred00X_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 10)': np.round(linear_pred00X[:-1])})
linear_pred00X_Confirmed_sub.to_csv('linear_pred00X_Confirmed_sub.csv', index = False)
linear_pred00X_Confirmed_sub

,Date,Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 10)
0,01/22/2020,-4560435.0
1,01/23/2020,-4345109.0
2,01/24/2020,-4135056.0
3,01/25/2020,-3930183.0
4,01/26/2020,-3730396.0
...,...,...
701,12/23/2021,309041659.0
702,12/24/2021,310255316.0
703,12/25/2021,311479848.0
704,12/26/2021,312715373.0


In [22]:
bayesian_pred00X_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 10)': np.round(bayesian_pred00X[:-1])})
bayesian_pred00X_Confirmed_sub.to_csv('bayesian_pred00X_Confirmed_sub.csv', index = False)
bayesian_pred00X_Confirmed_sub

,Date,Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 10)
0,01/22/2020,0.0
1,01/23/2020,5.0
2,01/24/2020,33.0
3,01/25/2020,102.0
4,01/26/2020,232.0
...,...,...
701,12/23/2021,226827584.0
702,12/24/2021,226552957.0
703,12/25/2021,226270436.0
704,12/26/2021,225979979.0


In [23]:
linear_pred00XX = linear_pred00X.reshape(1,-1)[0]
linear_pred00XX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 20)': np.round(linear_pred00XX[:-1])})
linear_pred00XX_Confirmed_sub.to_csv('linear_pred00X_Confirmed_sub.csv', index = False)
linear_pred00XX_Confirmed_sub

,Date,Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 20)
0,01/22/2020,-4560435.0
1,01/23/2020,-4345109.0
2,01/24/2020,-4135056.0
3,01/25/2020,-3930183.0
4,01/26/2020,-3730396.0
...,...,...
701,12/23/2021,309041659.0
702,12/24/2021,310255316.0
703,12/25/2021,311479848.0
704,12/26/2021,312715373.0


In [24]:
# Future predictions using SVM 
svm_pred00X_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'SVM Predicted # of Confirmed Cases Worldwide (Input 10)': np.round(svm_pred00X[:-1])})
svm_pred00X_Confirmed_sub.to_csv('svm_pred00X_Confirmed_sub.csv', index = False)
svm_pred00X_Confirmed_sub

,Date,SVM Predicted # of Confirmed Cases Worldwide (Input 10)
0,01/22/2020,12036222.0
1,01/23/2020,12036223.0
2,01/24/2020,12036231.0
3,01/25/2020,12036254.0
4,01/26/2020,12036298.0
...,...,...
701,12/23/2021,425698424.0
702,12/24/2021,427471260.0
703,12/25/2021,429249154.0
704,12/26/2021,431032113.0


In [25]:
linear_pred00XX = linear_pred00XX.reshape(1,-1)[0]
linear_pred00XX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 20)': np.round(linear_pred00XX[:-1])})
linear_pred00XX_Confirmed_sub.to_csv('linear_pred00X_Confirmed_sub.csv', index = False)
linear_pred00XX_Confirmed_sub

,Date,Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 20)
0,01/22/2020,-4560435.0
1,01/23/2020,-4345109.0
2,01/24/2020,-4135056.0
3,01/25/2020,-3930183.0
4,01/26/2020,-3730396.0
...,...,...
701,12/23/2021,309041659.0
702,12/24/2021,310255316.0
703,12/25/2021,311479848.0
704,12/26/2021,312715373.0


In [26]:
bayesian_pred00XX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 20)': np.round(bayesian_pred00XX[:-1])})
bayesian_pred00XX_Confirmed_sub.to_csv('bayesian_pred00X_Confirmed_sub.csv', index = False)
bayesian_pred00XX_Confirmed_sub

,Date,Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 20)
0,01/22/2020,0.0
1,01/23/2020,5.0
2,01/24/2020,33.0
3,01/25/2020,103.0
4,01/26/2020,234.0
...,...,...
701,12/23/2021,227324923.0
702,12/24/2021,227054714.0
703,12/25/2021,226776637.0
704,12/26/2021,226490651.0


In [27]:
# Future predictions using SVM 
svm_pred00XXX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'SVM Predicted # of Confirmed Cases Worldwide (Input 30)': np.round(svm_pred00XXX[:-1])})
svm_pred00XXX_Confirmed_sub.to_csv('svm_pred00X_Confirmed_sub.csv', index = False)
svm_pred00XXX_Confirmed_sub

,Date,SVM Predicted # of Confirmed Cases Worldwide (Input 30)
0,01/22/2020,13270809.0
1,01/23/2020,13270810.0
2,01/24/2020,13270819.0
3,01/25/2020,13270841.0
4,01/26/2020,13270885.0
...,...,...
701,12/23/2021,419305189.0
702,12/24/2021,421045334.0
703,12/25/2021,422790444.0
704,12/26/2021,424540525.0


In [28]:
linear_pred00XXX = linear_pred00XXX.reshape(1,-1)[0]
linear_pred00XXX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 30)': np.round(linear_pred00XXX[:-1])})
linear_pred00XXX_Confirmed_sub.to_csv('linear_pred00X_Confirmed_sub.csv', index = False)
linear_pred00XXX_Confirmed_sub

,Date,Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 30)
0,01/22/2020,-9916017.0
1,01/23/2020,-9572607.0
2,01/24/2020,-9236491.0
3,01/25/2020,-8907556.0
4,01/26/2020,-8585687.0
...,...,...
701,12/23/2021,316264876.0
702,12/24/2021,317630921.0
703,12/25/2021,319010097.0
704,12/26/2021,320402544.0


In [29]:
bayesian_pred00XXX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 30)': np.round(bayesian_pred00XXX[:-1])})
bayesian_pred00XXX_Confirmed_sub.to_csv('bayesian_pred00X_Confirmed_sub.csv', index = False)
bayesian_pred00XXX_Confirmed_sub

,Date,Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 30)
0,01/22/2020,0.0
1,01/23/2020,5.0
2,01/24/2020,34.0
3,01/25/2020,104.0
4,01/26/2020,236.0
...,...,...
701,12/23/2021,227815977.0
702,12/24/2021,227550142.0
703,12/25/2021,227276464.0
704,12/26/2021,226994903.0


In [30]:
# Future predictions using SVM 
svm_pred00LX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'SVM Predicted # of Confirmed Cases Worldwide (Input 40)': np.round(svm_pred00LX[:-1])})
svm_pred00LX_Confirmed_sub.to_csv('svm_pred00X_Confirmed_sub.csv', index = False)
svm_pred00LX_Confirmed_sub

,Date,SVM Predicted # of Confirmed Cases Worldwide (Input 40)
0,01/22/2020,13989830.0
1,01/23/2020,13989831.0
2,01/24/2020,13989839.0
3,01/25/2020,13989861.0
4,01/26/2020,13989904.0
...,...,...
701,12/23/2021,415536294.0
702,12/24/2021,417257205.0
703,12/25/2021,418983026.0
704,12/26/2021,420713764.0


In [31]:
linear_pred00LX = linear_pred00LX.reshape(1,-1)[0]
linear_pred00LX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 40)': np.round(linear_pred00LX[:-1])})
linear_pred00LX_Confirmed_sub.to_csv('linear_pred00X_Confirmed_sub.csv', index = False)
linear_pred00LX_Confirmed_sub

,Date,Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 40)
0,01/22/2020,-13299818.0
1,01/23/2020,-12883765.0
2,01/24/2020,-12476072.0
3,01/25/2020,-12076614.0
4,01/26/2020,-11685268.0
...,...,...
701,12/23/2021,318657516.0
702,12/24/2021,320079479.0
703,12/25/2021,321515465.0
704,12/26/2021,322965624.0


In [32]:
bayesian_pred00LX_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 40)': np.round(bayesian_pred00LX[:-1])})
bayesian_pred00LX_Confirmed_sub.to_csv('bayesian_pred00X_Confirmed_sub.csv', index = False)
bayesian_pred00LX_Confirmed_sub

,Date,Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 40)
0,01/22/2020,0.0
1,01/23/2020,6.0
2,01/24/2020,35.0
3,01/25/2020,108.0
4,01/26/2020,242.0
...,...,...
701,12/23/2021,228801889.0
702,12/24/2021,228544931.0
703,12/25/2021,228280184.0
704,12/26/2021,228007606.0


In [33]:
# Future predictions using SVM 
svm_pred00L_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'SVM Predicted # of Confirmed Cases Worldwide (Input 50)': np.round(svm_pred00L[:-1])})
svm_pred00L_Confirmed_sub.to_csv('svm_pred00X_Confirmed_sub.csv', index = False)
svm_pred00L_Confirmed_sub

,Date,SVM Predicted # of Confirmed Cases Worldwide (Input 50)
0,01/22/2020,14686223.0
1,01/23/2020,14686224.0
2,01/24/2020,14686232.0
3,01/25/2020,14686254.0
4,01/26/2020,14686297.0
...,...,...
701,12/23/2021,413344999.0
702,12/24/2021,415053535.0
703,12/25/2021,416766945.0
704,12/26/2021,418485236.0


In [34]:
linear_pred00L = linear_pred00L.reshape(1,-1)[0]
linear_pred00L_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 50)': np.round(linear_pred00L[:-1])})
linear_pred00L_Confirmed_sub.to_csv('linear_pred00X_Confirmed_sub.csv', index = False)
linear_pred00L_Confirmed_sub

,Date,Polynomial Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 50)
0,01/22/2020,-16914036.0
1,01/23/2020,-16424009.0
2,01/24/2020,-15943396.0
3,01/25/2020,-15472062.0
4,01/26/2020,-15009873.0
...,...,...
701,12/23/2021,320995137.0
702,12/24/2021,322470312.0
703,12/25/2021,323960351.0
704,12/26/2021,325465414.0


In [35]:
bayesian_pred00L_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 50)': np.round(bayesian_pred00L[:-1])})
bayesian_pred00L_Confirmed_sub.to_csv('bayesian_pred00L_Confirmed_sub.csv', index = False)
bayesian_pred00L_Confirmed_sub

,Date,Bayesian Ridge Regression Predictions Predicted # of Confirmed Cases Worldwide (Input 50)
0,01/22/2020,0.0
1,01/23/2020,6.0
2,01/24/2020,36.0
3,01/25/2020,110.0
4,01/26/2020,245.0
...,...,...
701,12/23/2021,229294405.0
702,12/24/2021,229041909.0
703,12/25/2021,228781649.0
704,12/26/2021,228513586.0


# Death Cases

In [36]:
X_train_deaths00X, X_test_deaths00X, y_train_deaths00X, y_test_deaths00X = train_test_split(days_since_1_22[10:], total_deaths[10:], test_size=0.12, shuffle=False)
# svm_deaths= svm_search.best_estimator_
svm_deaths00X = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00X.fit(X_train_deaths00X, y_train_deaths00X)
svm_pred00X = svm_deaths00X.predict(future_forcast)

# check against testing data
svm_test_pred00X = svm_deaths00X.predict(X_test_deaths00X)
print('MAE:', mean_absolute_error(svm_test_pred00X, y_test_deaths00X))
print('MSE:',mean_squared_error(svm_test_pred00X, y_test_deaths00X))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00X = poly.fit_transform(X_train_deaths00X)
poly_X_test_deaths00X = poly.fit_transform(X_test_deaths00X)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00X = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00X = bayesian_poly00X.fit_transform(X_train_deaths00X)
bayesian_poly_X_test_deaths00X = bayesian_poly00X.fit_transform(X_test_deaths00X)
bayesian_poly_future_forcast00X = bayesian_poly00X.fit_transform(future_forcast)

# polynomial regression
linear_model00X = LinearRegression(normalize=True, fit_intercept=False)
linear_model00X.fit(poly_X_train_deaths00X, y_train_deaths00X)
test_linear_pred00X = linear_model00X.predict(poly_X_test_deaths00X)
linear_pred00X = linear_model00X.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00X, y_test_deaths00X))
print('MSE:',mean_squared_error(test_linear_pred00X, y_test_deaths00X))

print(linear_model00X.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00X = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00X = BayesianRidge(fit_intercept=False)
bayesian_search00X = RandomizedSearchCV(bayesian00X, bayesian_grid00X, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00X.fit(bayesian_poly_X_train_deaths00X, y_train_deaths00X)

bayesian_search00X.best_params_

bayesian_deaths = bayesian_search00X.best_estimator_
test_bayesian_pred00X = bayesian_deaths.predict(bayesian_poly_X_test_deaths00X)
bayesian_pred00X = bayesian_deaths.predict(bayesian_poly_future_forcast00X)
print('MAE:', mean_absolute_error(test_bayesian_pred00X, y_test_deaths00X))
print('MSE:',mean_squared_error(test_bayesian_pred00X, y_test_deaths00X))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 2160795.842704933
MSE: 4969763738693.962
MAE: 298513.33441452583
MSE: 119455926306.39545
[[-1.39457451e+05  4.35201624e+03 -1.10874398e+01  6.49419774e-02
  -4.94633900e-05 -1.91339495e-08]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 165560.7547354965
MSE: 38490635773.198044


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.3s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
#10 deaths
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_deaths, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00X, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00X, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00X, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Death Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [38]:
X_train_deaths00XX, X_test_deaths00XX, y_train_deaths00XX, y_test_deaths00XX = train_test_split(days_since_1_22[20:], total_deaths[20:], test_size=0.12, shuffle=False)
# svm_deaths = svm_search.best_estimator_
svm_deaths00XX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00XX.fit(X_train_deaths00XX, y_train_deaths00XX)
svm_pred00XX = svm_deaths00XX.predict(future_forcast)

# check against testing data
svm_test_pred00XX = svm_deaths00XX.predict(X_test_deaths00XX)
print('MAE:', mean_absolute_error(svm_test_pred00XX, y_test_deaths00XX))
print('MSE:',mean_squared_error(svm_test_pred00XX, y_test_deaths00XX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00XX = poly.fit_transform(X_train_deaths00XX)
poly_X_test_deaths00XX = poly.fit_transform(X_test_deaths00XX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00XX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00XX = bayesian_poly00XX.fit_transform(X_train_deaths00XX)
bayesian_poly_X_test_deaths00XX = bayesian_poly00XX.fit_transform(X_test_deaths00XX)
bayesian_poly_future_forcast00XX = bayesian_poly00XX.fit_transform(future_forcast)

# polynomial regression
linear_model00XX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00XX.fit(poly_X_train_deaths00XX, y_train_deaths00XX)
test_linear_pred00XX = linear_model00X.predict(poly_X_test_deaths00XX)
linear_pred00XX = linear_model00X.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00XX, y_test_deaths00XX))
print('MSE:',mean_squared_error(test_linear_pred00XX, y_test_deaths00XX))

print(linear_model00XX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00XX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00XX = BayesianRidge(fit_intercept=False)
bayesian_search00XX = RandomizedSearchCV(bayesian00XX, bayesian_grid00XX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00XX.fit(bayesian_poly_X_train_deaths00XX, y_train_deaths00XX)

bayesian_search00XX.best_params_

bayesian_deaths = bayesian_search00XX.best_estimator_
test_bayesian_pred00XX = bayesian_deaths.predict(bayesian_poly_X_test_deaths00XX)
bayesian_pred00XX = bayesian_deaths.predict(bayesian_poly_future_forcast00XX)
print('MAE:', mean_absolute_error(test_bayesian_pred00XX, y_test_deaths00XX))
print('MSE:',mean_squared_error(test_bayesian_pred00XX, y_test_deaths00XX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 2133239.393477698
MSE: 4839226130982.811
MAE: 301278.03091326903
MSE: 120849822716.50372
[[-2.37511638e+05  6.84163201e+03 -3.15160358e+01  1.37477378e-01
  -1.65444543e-04  4.93855623e-08]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 165690.0472874342
MSE: 38399386759.39746


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [39]:
#20 deaths
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_deaths, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00XX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00XX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00XX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Death Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [40]:
X_train_deaths00XXX, X_test_deaths00XXX, y_train_deaths00XXX, y_test_deaths00XXX = train_test_split(days_since_1_22[30:], total_deaths[30:], test_size=0.12, shuffle=False)
# svm_deaths = svm_search.best_estimator_
svm_deaths00XXX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00XXX.fit(X_train_deaths00XXX, y_train_deaths00XXX)
svm_pred00XXX = svm_deaths00XXX.predict(future_forcast)

# check against testing data
svm_test_pred00XXX = svm_deaths00XXX.predict(X_test_deaths00XXX)
print('MAE:', mean_absolute_error(svm_test_pred00XXX, y_test_deaths00XXX))
print('MSE:',mean_squared_error(svm_test_pred00XXX, y_test_deaths00XXX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00XXX = poly.fit_transform(X_train_deaths00XXX)
poly_X_test_deaths00XXX = poly.fit_transform(X_test_deaths00XXX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00XXX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00XXX = bayesian_poly00XXX.fit_transform(X_train_deaths00XXX)
bayesian_poly_X_test_deaths00XXX = bayesian_poly00XXX.fit_transform(X_test_deaths00XXX)
bayesian_poly_future_forcast00XXX = bayesian_poly00XXX.fit_transform(future_forcast)

# polynomial regression
linear_model00XXX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00XXX.fit(poly_X_train_deaths00XXX, y_train_deaths00XXX)
test_linear_pred00XXX = linear_model00X.predict(poly_X_test_deaths00XXX)
linear_pred00XXX = linear_model00XXX.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00XXX, y_test_deaths00XXX))
print('MSE:',mean_squared_error(test_linear_pred00XXX, y_test_deaths00XXX))

print(linear_model00XXX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00XXX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00XXX = BayesianRidge(fit_intercept=False)
bayesian_search00XXX = RandomizedSearchCV(bayesian00XXX, bayesian_grid00XXX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00XXX.fit(bayesian_poly_X_train_deaths00XXX, y_train_deaths00XXX)

bayesian_search00XXX.best_params_

bayesian_deaths = bayesian_search00XXX.best_estimator_
test_bayesian_pred00XXX = bayesian_deaths.predict(bayesian_poly_X_test_deaths00XXX)
bayesian_pred00XXX = bayesian_deaths.predict(bayesian_poly_future_forcast00XXX)
print('MAE:', mean_absolute_error(test_bayesian_pred00XXX, y_test_deaths00XXX))
print('MSE:',mean_squared_error(test_bayesian_pred00XXX, y_test_deaths00XXX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 2077480.3400895875
MSE: 4589043327091.101
MAE: 304053.6711704275
MSE: 122269638041.31087
[[-3.81370990e+05  1.02360833e+04 -5.81712936e+01  2.29330770e-01
  -3.09107352e-04  1.32806235e-07]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 168355.80214857418
MSE: 39415526853.31082


In [41]:
#30 deaths
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_deaths, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00XXX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00XXX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00XXX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Death Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [42]:
X_train_deaths00LX, X_test_deaths00LX, y_train_deaths00LX, y_test_deaths00LX = train_test_split(days_since_1_22[40:], total_deaths[40:], test_size=0.12, shuffle=False)
# svm_deaths = svm_search.best_estimator_
svm_deaths00LX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00LX.fit(X_train_deaths00LX, y_train_deaths00LX)
svm_pred00LX = svm_deaths00LX.predict(future_forcast)

# check against testing data
svm_test_pred00LX = svm_deaths00LX.predict(X_test_deaths00LX)
print('MAE:', mean_absolute_error(svm_test_pred00LX, y_test_deaths00LX))
print('MSE:',mean_squared_error(svm_test_pred00LX, y_test_deaths00LX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00LX = poly.fit_transform(X_train_deaths00LX)
poly_X_test_deaths00LX = poly.fit_transform(X_test_deaths00LX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00LX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00LX = bayesian_poly00LX.fit_transform(X_train_deaths00LX)
bayesian_poly_X_test_deaths00LX = bayesian_poly00LX.fit_transform(X_test_deaths00LX)
bayesian_poly_future_forcast00LX = bayesian_poly00LX.fit_transform(future_forcast)

# polynomial regression
linear_model00LX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00LX.fit(poly_X_train_deaths00LX, y_train_deaths00LX)
test_linear_pred00LX = linear_model00LX.predict(poly_X_test_deaths00LX)
linear_pred00LX = linear_model00LX.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00LX, y_test_deaths00LX))
print('MSE:',mean_squared_error(test_linear_pred00LX, y_test_deaths00LX))

print(linear_model00LX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00LX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00LX = BayesianRidge(fit_intercept=False)
bayesian_search00LX = RandomizedSearchCV(bayesian00LX, bayesian_grid00LX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00LX.fit(bayesian_poly_X_train_deaths00LX, y_train_deaths00LX)

bayesian_search00LX.best_params_

bayesian_deaths = bayesian_search00LX.best_estimator_
test_bayesian_pred00LX = bayesian_deaths.predict(bayesian_poly_X_test_deaths00LX)
bayesian_pred00LX = bayesian_deaths.predict(bayesian_poly_future_forcast00LX)
print('MAE:', mean_absolute_error(test_bayesian_pred00LX, y_test_deaths00LX))
print('MSE:',mean_squared_error(test_bayesian_pred00LX, y_test_deaths00LX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 2044598.0185368997
MSE: 4435051691320.053
MAE: 57621.05809491014
MSE: 3871018717.316749
[[-5.75546036e+05  1.45465139e+04 -9.07878092e+01  3.39019771e-01
  -4.77746756e-04  2.29490979e-07]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 172757.88946840764
MSE: 41096329336.8908


In [43]:
#40 deaths
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_deaths, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00LX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00LX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00LX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Death Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [44]:
X_train_deaths00L, X_test_deaths00L, y_train_deaths00L, y_test_deaths00L = train_test_split(days_since_1_22[50:], total_deaths[50:], test_size=0.12, shuffle=False)
# svm_deaths = svm_search.best_estimator_
svm_deaths00L = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00L.fit(X_train_deaths00L, y_train_deaths00L)
svm_pred00L = svm_deaths00L.predict(future_forcast)

# check against testing data
svm_test_pred00L = svm_deaths00L.predict(X_test_deaths00L)
print('MAE:', mean_absolute_error(svm_test_pred00L, y_test_deaths00L))
print('MSE:',mean_squared_error(svm_test_pred00L, y_test_deaths00L))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00L = poly.fit_transform(X_train_deaths00L)
poly_X_test_deaths00L = poly.fit_transform(X_test_deaths00L)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00L = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00L = bayesian_poly00L.fit_transform(X_train_deaths00L)
bayesian_poly_X_test_deaths00L = bayesian_poly00L.fit_transform(X_test_deaths00L)
bayesian_poly_future_forcast00L = bayesian_poly00L.fit_transform(future_forcast)

# polynomial regression
linear_model00L = LinearRegression(normalize=True, fit_intercept=False)
linear_model00L.fit(poly_X_train_deaths00L, y_train_deaths00L)
test_linear_pred00L = linear_model00L.predict(poly_X_test_deaths00L)
linear_pred00L = linear_model00L.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00L, y_test_deaths00L))
print('MSE:',mean_squared_error(test_linear_pred00L, y_test_deaths00L))

print(linear_model00L.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00L = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00L = BayesianRidge(fit_intercept=False)
bayesian_search00L = RandomizedSearchCV(bayesian00L, bayesian_grid00L, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00L.fit(bayesian_poly_X_train_deaths00L, y_train_deaths00L)

bayesian_search00L.best_params_

bayesian_deaths = bayesian_search00L.best_estimator_
test_bayesian_pred00L = bayesian_deaths.predict(bayesian_poly_X_test_deaths00L)
bayesian_pred00L = bayesian_deaths.predict(bayesian_poly_future_forcast00L)
print('MAE:', mean_absolute_error(test_bayesian_pred00L, y_test_deaths00L))
print('MSE:',mean_squared_error(test_bayesian_pred00L, y_test_deaths00L))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 2026003.620041893
MSE: 4349610891600.7773
MAE: 23462.774643313332
MSE: 920063495.0150778
[[-8.01600147e+05  1.92737405e+04 -1.25099206e+02  4.50860555e-01
  -6.45470956e-04  3.23681888e-07]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 183198.82474872423
MSE: 45732986326.02896


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:
#50 deaths
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_deaths, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00L, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00L, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00L, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Death Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [46]:
# Future predictions using SVM 
svm_pred00X_Confirmed_sub = pd.DataFrame({'Date': future_forcast_dates[:-1], 'SVM Predicted # of Confirmed Cases Worldwide (Input 10)': np.round(svm_pred00X[:-1])})
svm_pred00X_Confirmed_sub.to_csv('svm_pred00X_Confirmed_sub.csv', index = False)
svm_pred00X_Confirmed_sub

,Date,SVM Predicted # of Confirmed Cases Worldwide (Input 10)
0,01/22/2020,633466.0
1,01/23/2020,633466.0
2,01/24/2020,633467.0
3,01/25/2020,633467.0
4,01/26/2020,633468.0
...,...,...
701,12/23/2021,8668783.0
702,12/24/2021,8703220.0
703,12/25/2021,8737755.0
704,12/26/2021,8772389.0


# Recovered Cases

In [47]:
X_train_deaths00X, X_test_deaths00X, y_train_deaths00X, y_test_deaths00X = train_test_split(days_since_1_22[10:], total_recovered[10:], test_size=0.12, shuffle=False)
# svm_deaths= svm_search.best_estimator_
svm_deaths00X = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00X.fit(X_train_deaths00X, y_train_deaths00X)
svm_pred00X = svm_deaths00X.predict(future_forcast)

# check against testing data
svm_test_pred00X = svm_deaths00X.predict(X_test_deaths00X)
print('MAE:', mean_absolute_error(svm_test_pred00X, y_test_deaths00X))
print('MSE:',mean_squared_error(svm_test_pred00X, y_test_deaths00X))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00X = poly.fit_transform(X_train_deaths00X)
poly_X_test_deaths00X = poly.fit_transform(X_test_deaths00X)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00X = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00X = bayesian_poly00X.fit_transform(X_train_deaths00X)
bayesian_poly_X_test_deaths00X = bayesian_poly00X.fit_transform(X_test_deaths00X)
bayesian_poly_future_forcast00X = bayesian_poly00X.fit_transform(future_forcast)

# polynomial regression
linear_model00X = LinearRegression(normalize=True, fit_intercept=False)
linear_model00X.fit(poly_X_train_deaths00X, y_train_deaths00X)
test_linear_pred00X = linear_model00X.predict(poly_X_test_deaths00X)
linear_pred00X = linear_model00X.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00X, y_test_deaths00X))
print('MSE:',mean_squared_error(test_linear_pred00X, y_test_deaths00X))

print(linear_model00X.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00X = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00X = BayesianRidge(fit_intercept=False)
bayesian_search00X = RandomizedSearchCV(bayesian00X, bayesian_grid00X, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00X.fit(bayesian_poly_X_train_deaths00X, y_train_deaths00X)

bayesian_search00X.best_params_

bayesian_deaths = bayesian_search00X.best_estimator_
test_bayesian_pred00X = bayesian_deaths.predict(bayesian_poly_X_test_deaths00X)
bayesian_pred00X = bayesian_deaths.predict(bayesian_poly_future_forcast00X)
print('MAE:', mean_absolute_error(test_bayesian_pred00X, y_test_deaths00X))
print('MSE:',mean_squared_error(test_bayesian_pred00X, y_test_deaths00X))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 229096414.01748887
MSE: 5.3071295528170616e+16
MAE: 280415894.54411876
MSE: 1.0412149486512128e+17
[[ 1.44131190e+07 -7.14114674e+05  9.20045095e+03 -4.44817491e+01
   9.73641251e-02 -7.60750845e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 56570120.4014384
MSE: 4924602418625302.0


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
#10 deaths
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_recovered, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00X, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00X, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00X, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Recovered Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [49]:
X_train_deaths00XX, X_test_deaths00XX, y_train_deaths00XX, y_test_deaths00XX = train_test_split(days_since_1_22[20:], total_recovered[20:], test_size=0.12, shuffle=False)
# svm_deaths = svm_search.best_estimator
svm_deaths00XX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00XX.fit(X_train_deaths00XX, y_train_deaths00XX)
svm_pred00XX = svm_deaths00XX.predict(future_forcast)

# check against testing data
svm_test_pred00XX = svm_deaths00XX.predict(X_test_deaths00XX)
print('MAE:', mean_absolute_error(svm_test_pred00XX, y_test_deaths00XX))
print('MSE:',mean_squared_error(svm_test_pred00XX, y_test_deaths00XX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00XX = poly.fit_transform(X_train_deaths00XX)
poly_X_test_deaths00XX = poly.fit_transform(X_test_deaths00XX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00XX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00XX = bayesian_poly00XX.fit_transform(X_train_deaths00XX)
bayesian_poly_X_test_deaths00XX = bayesian_poly00XX.fit_transform(X_test_deaths00XX)
bayesian_poly_future_forcast00XX = bayesian_poly00XX.fit_transform(future_forcast)

# polynomial regression
linear_model00XX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00XX.fit(poly_X_train_deaths00XX, y_train_deaths00XX)
test_linear_pred00XX = linear_model00X.predict(poly_X_test_deaths00XX)
linear_pred00XX = linear_model00X.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00XX, y_test_deaths00XX))
print('MSE:',mean_squared_error(test_linear_pred00XX, y_test_deaths00XX))

print(linear_model00XX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00XX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00XX = BayesianRidge(fit_intercept=False)
bayesian_search00XX = RandomizedSearchCV(bayesian00XX, bayesian_grid00XX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00XX.fit(bayesian_poly_X_train_deaths00XX, y_train_deaths00XX)

bayesian_search00XX.best_params_

bayesian_deaths = bayesian_search00XX.best_estimator_
test_bayesian_pred00XX = bayesian_deaths.predict(bayesian_poly_X_test_deaths00XX)
bayesian_pred00XX = bayesian_deaths.predict(bayesian_poly_future_forcast00XX)
print('MAE:', mean_absolute_error(test_bayesian_pred00XX, y_test_deaths00XX))
print('MSE:',mean_squared_error(test_bayesian_pred00XX, y_test_deaths00XX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 228203678.55936506
MSE: 5.264005788107618e+16
MAE: 283192865.7426738
MSE: 1.0535739433142526e+17
[[ 2.03942692e+07 -8.48208786e+05  1.01482960e+04 -4.73008400e+01
   1.01001214e-01 -7.77144451e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 57494463.35540896
MSE: 5036261946781332.0


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
#20 deaths
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_recovered, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00XX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00XX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00XX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Recovered Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [51]:
X_train_deaths00XXX, X_test_deaths00XXX, y_train_deaths00XXX, y_test_deaths00XXX = train_test_split(days_since_1_22[30:], total_recovered[30:], test_size=0.12, shuffle=False)
# svm_recovered = svm_search.best_estimator_
svm_deaths00XXX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00XXX.fit(X_train_deaths00XXX, y_train_deaths00XXX)
svm_pred00XXX = svm_deaths00XXX.predict(future_forcast)

# check against testing data
svm_test_pred00XXX = svm_deaths00XXX.predict(X_test_deaths00XXX)
print('MAE:', mean_absolute_error(svm_test_pred00XXX, y_test_deaths00XXX))
print('MSE:',mean_squared_error(svm_test_pred00XXX, y_test_deaths00XXX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00XXX = poly.fit_transform(X_train_deaths00XXX)
poly_X_test_deaths00XXX = poly.fit_transform(X_test_deaths00XXX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00XXX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00XXX = bayesian_poly00XXX.fit_transform(X_train_deaths00XXX)
bayesian_poly_X_test_deaths00XXX = bayesian_poly00XXX.fit_transform(X_test_deaths00XXX)
bayesian_poly_future_forcast00XXX = bayesian_poly00XXX.fit_transform(future_forcast)

# polynomial regression
linear_model00XXX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00XXX.fit(poly_X_train_deaths00XXX, y_train_deaths00XXX)
test_linear_pred00XXX = linear_model00X.predict(poly_X_test_deaths00XXX)
linear_pred00XXX = linear_model00XXX.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00XXX, y_test_deaths00XXX))
print('MSE:',mean_squared_error(test_linear_pred00XXX, y_test_deaths00XXX))

print(linear_model00XXX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00XXX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00XXX = BayesianRidge(fit_intercept=False)
bayesian_search00XXX = RandomizedSearchCV(bayesian00XXX, bayesian_grid00XXX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00XXX.fit(bayesian_poly_X_train_deaths00XXX, y_train_deaths00XXX)

bayesian_search00XXX.best_params_

bayesian_deaths = bayesian_search00XXX.best_estimator_
test_bayesian_pred00XXX = bayesian_deaths.predict(bayesian_poly_X_test_deaths00XXX)
bayesian_pred00XXX = bayesian_deaths.predict(bayesian_poly_future_forcast00XXX)
print('MAE:', mean_absolute_error(test_bayesian_pred00XXX, y_test_deaths00XXX))
print('MSE:',mean_squared_error(test_bayesian_pred00XXX, y_test_deaths00XXX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 226704817.2880892
MSE: 5.19331662859801e+16
MAE: 285993162.79320866
MSE: 1.066188752227071e+17
[[ 2.66558317e+07 -9.76000853e+05  1.09843160e+04 -4.95886656e+01
   1.03643794e-01 -7.87057215e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 58403495.11996837
MSE: 5142473074618492.0


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
#30 recovered
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_recovered, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00XXX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00XXX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00XXX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Recovered Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [53]:
X_train_deaths00LX, X_test_deaths00LX, y_train_deaths00LX, y_test_deaths00LX = train_test_split(days_since_1_22[40:], total_recovered[40:], test_size=0.12, shuffle=False)
# svm_recovered = svm_search.best_estimator_
svm_deaths00LX = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00LX.fit(X_train_deaths00LX, y_train_deaths00LX)
svm_pred00LX = svm_deaths00LX.predict(future_forcast)

# check against testing data
svm_test_pred00LX = svm_deaths00LX.predict(X_test_deaths00LX)
print('MAE:', mean_absolute_error(svm_test_pred00LX, y_test_deaths00LX))
print('MSE:',mean_squared_error(svm_test_pred00LX, y_test_deaths00LX))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00LX = poly.fit_transform(X_train_deaths00LX)
poly_X_test_deaths00LX = poly.fit_transform(X_test_deaths00LX)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00LX = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00LX = bayesian_poly00LX.fit_transform(X_train_deaths00LX)
bayesian_poly_X_test_deaths00LX = bayesian_poly00LX.fit_transform(X_test_deaths00LX)
bayesian_poly_future_forcast00LX = bayesian_poly00LX.fit_transform(future_forcast)

# polynomial regression
linear_model00LX = LinearRegression(normalize=True, fit_intercept=False)
linear_model00LX.fit(poly_X_train_deaths00LX, y_train_deaths00LX)
test_linear_pred00LX = linear_model00LX.predict(poly_X_test_deaths00LX)
linear_pred00LX = linear_model00LX.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00LX, y_test_deaths00LX))
print('MSE:',mean_squared_error(test_linear_pred00LX, y_test_deaths00LX))

print(linear_model00LX.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00LX = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00LX = BayesianRidge(fit_intercept=False)
bayesian_search00LX = RandomizedSearchCV(bayesian00LX, bayesian_grid00LX, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00LX.fit(bayesian_poly_X_train_deaths00LX, y_train_deaths00LX)

bayesian_search00LX.best_params_

bayesian_deaths = bayesian_search00LX.best_estimator_
test_bayesian_pred00LX = bayesian_deaths.predict(bayesian_poly_X_test_deaths00LX)
bayesian_pred00LX = bayesian_deaths.predict(bayesian_poly_future_forcast00LX)
print('MAE:', mean_absolute_error(test_bayesian_pred00LX, y_test_deaths00LX))
print('MSE:',mean_squared_error(test_bayesian_pred00LX, y_test_deaths00LX))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 225625887.7030265
MSE: 5.140916491779085e+16
MAE: 264891107.52207166
MSE: 9.119795691627394e+16
[[ 2.97987710e+07 -1.00065774e+06  1.08035538e+04 -4.77098440e+01
   9.87841853e-02 -7.47907907e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 60182654.60563601
MSE: 5338887689018253.0


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [54]:
#40 Recovered
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_recovered, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00LX, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00LX, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00LX, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Recovered Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>

In [55]:
X_train_deaths00L, X_test_deaths00L, y_train_deaths00L, y_test_deaths00L = train_test_split(days_since_1_22[50:], total_recovered[50:], test_size=0.12, shuffle=False)
# svm_recovered = svm_search.best_estimator_
svm_deaths00L = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_deaths00L.fit(X_train_deaths00L, y_train_deaths00L)
svm_pred00L = svm_deaths00L.predict(future_forcast)

# check against testing data
svm_test_pred00L = svm_deaths00L.predict(X_test_deaths00L)
print('MAE:', mean_absolute_error(svm_test_pred00L, y_test_deaths00L))
print('MSE:',mean_squared_error(svm_test_pred00L, y_test_deaths00L))

# transform our data for polynomial regression
poly = PolynomialFeatures(degree=5)
poly_X_train_deaths00L = poly.fit_transform(X_train_deaths00L)
poly_X_test_deaths00L = poly.fit_transform(X_test_deaths00L)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly00L = PolynomialFeatures(degree=4)
bayesian_poly_X_train_deaths00L = bayesian_poly00L.fit_transform(X_train_deaths00L)
bayesian_poly_X_test_deaths00L = bayesian_poly00L.fit_transform(X_test_deaths00L)
bayesian_poly_future_forcast00L = bayesian_poly00L.fit_transform(future_forcast)

# polynomial regression
linear_model00L = LinearRegression(normalize=True, fit_intercept=False)
linear_model00L.fit(poly_X_train_deaths00L, y_train_deaths00L)
test_linear_pred00L = linear_model00L.predict(poly_X_test_deaths00L)
linear_pred00L = linear_model00L.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred00L, y_test_deaths00L))
print('MSE:',mean_squared_error(test_linear_pred00L, y_test_deaths00L))

print(linear_model00L.coef_)


# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid00L = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian00L = BayesianRidge(fit_intercept=False)
bayesian_search00L = RandomizedSearchCV(bayesian00L, bayesian_grid00L, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search00L.fit(bayesian_poly_X_train_deaths00L, y_train_deaths00L)

bayesian_search00L.best_params_

bayesian_deaths = bayesian_search00L.best_estimator_
test_bayesian_pred00L = bayesian_deaths.predict(bayesian_poly_X_test_deaths00L)
bayesian_pred00L = bayesian_deaths.predict(bayesian_poly_future_forcast00L)
print('MAE:', mean_absolute_error(test_bayesian_pred00L, y_test_deaths00L))
print('MSE:',mean_squared_error(test_bayesian_pred00L, y_test_deaths00L))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


MAE: 225026182.36981156
MSE: 5.111967199183856e+16
MAE: 259711727.0513957
MSE: 8.724725108906456e+16
[[ 3.34146602e+07 -1.05050368e+06  1.09592352e+04 -4.75124965e+01
   9.74038492e-02 -7.33982306e-05]]
Fitting 3 folds for each of 40 candidates, totalling 120 fits
MAE: 61051164.560014606
MSE: 5428383012907734.0


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [56]:
#50 Recovered
%matplotlib notebook
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20, 12))
plt.plot(adjusted_dates, total_recovered, linestyle='solid', color='black')
plt.plot(future_forcast, svm_pred00L, linestyle='dotted', color='purple')
plt.plot(future_forcast, linear_pred00L, linestyle='dotted', color='orange')
plt.plot(future_forcast, bayesian_pred00L, linestyle='dotted', color='green')
plt.title('# of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Recovered Cases', 'SVM predictions', 'Polynomial Regression Predictions', 'Bayesian Ridge Regression Predictions'])
plt.xticks(size=20)
plt.show()

<IPython.core.display.Javascript object>